In [3]:
# load in packages
from itertools import combinations

#from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('C://users/Administrator/Desktop/training.csv')
train_data.head(10)

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2
5,9,No,0,1,41.189415,-0.905350,2,4,4,1
6,11,No,0,1,40.690409,1.085939,2,2,1,2
7,14,No,0,2,25.971529,-1.424817,1,3,3,2
8,15,Yes,0,2,31.930423,0.393317,2,3,1,2
9,16,No,0,1,27.474650,0.566472,2,2,3,2


In [4]:
#x and y axes input data
y = train_data['Promotion']
x =  train_data.drop(['Promotion'], axis = 1)

In [5]:
df = train_data.drop(['ID','purchase','Promotion'], axis = 1)

In [6]:
y

0         No
1         No
2         No
3         No
4        Yes
        ... 
84529     No
84530    Yes
84531     No
84532     No
84533     No
Name: Promotion, Length: 84534, dtype: object

In [21]:
#Converting YES and No to numbers for XGBoost preparation
Promotion = []

for i in y: 
    if i == 'Yes': 
       Promotion.append(1) 
    elif i == 'No': 
       Promotion.append(0)
df['Promotion'] = Promotion

In [22]:
df

,V1,V2,V3,V4,V5,V6,V7,Promotion
0,2,30.443518,-1.165083,1,1,3,2,0
1,3,32.159350,-0.645617,2,3,2,2,0
2,2,30.431659,0.133583,1,1,4,2,0
3,0,26.588914,-0.212728,2,1,4,2,0
4,3,28.044332,-0.385883,1,1,2,2,1
...,...,...,...,...,...,...,...,...
84529,1,30.084876,1.345672,1,1,3,1,0
84530,3,33.501485,-0.299306,1,1,4,1,1
84531,1,31.492019,1.085939,2,3,2,2,0
84532,1,37.766106,0.999361,2,2,1,2,0


In [23]:
# Split data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, Promotion, test_size=0.2)

Training XGBoost Model

In [29]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
xgbr = xgb.XGBClassifier()
param = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=1000, 
 gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

In [30]:
gsearch.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:39:24] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=0, gpu_id=None,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=1000, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=1, seed=27, subsample=0.8,
                                     tree_

In [31]:
print("Best params:{}".format(gsearch.best_params_))

Best params:{'max_depth': 3, 'min_child_weight': 1}


In [32]:
gsearch.best_score_

1.0

In [33]:
gsearch.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=27, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=27, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [34]:
y_val_pred_x = gsearch.predict(X_test)

y_pred_temp = [0 if y_val_pred_x[i] <= 0.5 else 1 for i in range(len(y_val_pred_x))]

In [37]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_val_pred_x>0.5)
print ("SCORE:", accuracy)

SCORE: 1.0


Predictions of promotions

In [41]:
Promo = []

for i in y_val_pred_x: 
    if i == 1: 
       Promo.append('Yes') 
    elif i == 0: 
       Promo.append('No') 
y['Promo'] = Promo

In [42]:
Promo[0]

'No'

In [43]:
Promo

['No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes'